# Sakamoto LM

## Install Fairseq

https://github.com/pytorch/fairseq#requirements-and-installation

In [ ]:
!mkdir tmp
%cd tmp

/content/tmp


In [ ]:
!git clone https://github.com/pytorch/fairseq
%cd fairseq
!pip install --editable ./
%cd ..

Cloning into 'fairseq'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 19597 (delta 18), reused 24 (delta 10), pack-reused 19544
Receiving objects: 100% (19597/19597), 8.85 MiB | 24.97 MiB/s, done.
Resolving deltas: 100% (14596/14596), done.
/content/tmp/fairseq
Obtaining file:///content/tmp/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 71kB 3.4MB/s 
     |████████████████████████████████| 122kB 12.3MB/s 
     |████████████████████████████████| 112kB 22.5MB/s 
     |████████████████████████████████| 276kB 13.4MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp36-none-any.whl size=141230 sha256=da611d63804bba21c0666927cdcbed8bb0d1cb952ec7fd271f333caca928600f
  Stored in directory: 

In [ ]:
%cd ..

/content


## Train a model

https://github.com/pytorch/fairseq/tree/master/examples/language_model#training-a-transformer-language-model-with-the-cli-tools

### Preprocess the data

In [ ]:
!cp drive/My\ Drive/sksk_sskn.tokens .

In [ ]:
!sed -n 1,2000p sksk_sskn.tokens > sksk_sskn.train.tokens
!sed -n 2001,2250p sksk_sskn.tokens > sksk_sskn.valid.tokens
!sed -n 2251,2500p sksk_sskn.tokens > sksk_sskn.test.tokens

In [ ]:
!fairseq-preprocess \
    --only-source \
    --trainpref sksk_sskn.train.tokens \
    --validpref sksk_sskn.valid.tokens \
    --testpref sksk_sskn.test.tokens \
    --destdir data-bin/sksk_sskn \
    --workers 20

2020-11-09 06:36:49 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='data-bin/sksk_sskn', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=True, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang=None, srcdict=None, target_lang=None, task='translation', tensorboard_logdir=None, testpref='sksk_sskn.test.tokens', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, tpu=False, trainpref='sksk_sskn.train.tokens', user_d

### Train a language model

https://github.com/pytorch/fairseq/blob/master/examples/roberta/README.pretraining.md#2-train-roberta-base

https://fairseq.readthedocs.io/en/latest/command_line_tools.html#fairseq-train
https://fairseq.readthedocs.io/en/latest/tasks.html#language-modeling

In [ ]:
!fairseq-train --task language_modeling \
    data-bin/sksk_sskn \
    --save-dir checkpoints/transformer_sakamoto \
    --arch transformer_lm --share-decoder-input-output-embed \
    --dropout 0.1 \
    --optimizer adam --adam-betas '(0.9, 0.98)' --weight-decay 0.01 --clip-norm 0.0 \
    --lr 0.0005 --lr-scheduler inverse_sqrt --warmup-updates 4000 --warmup-init-lr 1e-07 \
    --tokens-per-sample 64 --sample-break-mode eos \
    --max-sentences 16 --update-freq 2 \
    --fp16 \
    --max-update 50000 \
    --no-epoch-checkpoints

ストリーミング出力は最後の 5000 行に切り捨てられました。
2020-11-09 07:16:10 | INFO | fairseq_cli.train | end of epoch 343 (average epoch stats below)
2020-11-09 07:16:10 | INFO | train | epoch 343 | loss 0.824 | ppl 1.77 | wps 3377.3 | ups 7.61 | wpb 443.8 | bsz 31.7 | num_updates 21608 | lr 0.000215126 | gnorm 0.446 | loss_scale 256 | train_wall 4 | wall 2345
epoch 344:   0% 0/63 [00:00<?, ?it/s]2020-11-09 07:16:10 | INFO | fairseq.trainer | begin training epoch 344
epoch 344:  97% 61/63 [00:04<00:00, 17.12it/s]2020-11-09 07:16:14 | INFO | fairseq_cli.train | begin validation on "valid" subset

epoch 344 | valid on 'valid' subset:   0% 0/16 [00:00<?, ?it/s]
epoch 344 | valid on 'valid' subset:  12% 2/16 [00:00<00:00, 16.23it/s]
epoch 344 | valid on 'valid' subset:  75% 12/16 [00:00<00:00, 21.67it/s]
                                                                        2020-11-09 07:16:14 | INFO | valid | epoch 344 | valid on 'valid' subset | loss 15.476 | ppl 45582.1 | wps 25996.7 | wpb 273.5 | bsz 15.6 | 

### Evaluate

In [ ]:
!fairseq-eval-lm data-bin/sksk_sskn \
    --path checkpoints/transformer_sakamoto/checkpoint_best.pt \
    --batch-size 2 \
    --tokens-per-sample 64 \
    --context-window 0

2020-11-09 08:06:42 | INFO | fairseq_cli.eval_lm | {'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'tensorboard_logdir': None, 'wandb_project': None, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False}, 'common_eval': {'_name': None, 'path': 'checkpoints/transformer_sakamoto/checkpoint_best.pt', 'post_process': None, 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_rank': 0, 'distributed_backend': 'nccl', 'distributed_init_method': None, 'distributed_por

## Sample from the model

https://github.com/pytorch/fairseq/tree/master/examples/language_model#example-usage

In [ ]:
bos_set = set()
with open('sksk_sskn.train.tokens') as f:
    for line in f:
        bos_set.add(line.split()[0])

In [ ]:
import sys
sys.path.append('/content/tmp/fairseq')

In [ ]:
from fairseq.models.transformer_lm import TransformerLanguageModel
sakamoto_lm = TransformerLanguageModel.from_pretrained('checkpoints/transformer_sakamoto', 'checkpoint_best.pt', data_name_or_path='data-bin/sksk_sskn')

In [ ]:
sakamoto_lm.sample('ざむ', beam=5, sampling=True, sampling_topk=10, temperature=0.8)

'ざむ ちゃん の 話 を して いる'